In [8]:
!pip install llama-index-readers-nougat-ocr

ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\mateo\\Desktop\\Semestre 8\\IA2\\Exposition\\my_env\\Lib\\site-packages\\cv2\\cv2.pyd'
Check the permissions.


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached llama_index_readers_nougat_ocr-0.3.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached nougat_ocr-0.1.17-py3-none-any.whl.metadata (10 kB)
  Using cached timm-0.5.4-py3-none-any.whl.metadata (36 kB)
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached lightning-2.5.1-py3-none-any.whl.metadata (39 kB)
  Using cached python_levenshtein-0.27.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached sconf-0.2.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached albumentations-2.0.5-py3-none-any.whl.metadata (41 kB)
  Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached albucore-0.0.23-py3-none-any.whl.metadata (5.3 kB)
  Using cached lightning_utilities-0.14.3-py3-none-any.whl.metadata (5.6 kB)
  Using cached torchmetrics-1.7.1-py3-none-any.whl.metadata (21 kB)
  Using cached pytorch_lightning-2.5.1-py3-none-any.whl.metadata (20 kB

In [3]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
#from llama_index import (ServiceContext, KnowledgeGraphIndex)
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core import KnowledgeGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.storage_context import StorageContext
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from pyvis.network import Network
from IPython.display import display, HTML
from dotenv import load_dotenv
import os
import pandas as pd
from llama_index.core import Document
from llama_index.core import Settings
from llama_index.readers.pdf_table import PDFTableReader
from pathlib import Path
from llama_index.readers.pdf_marker import PDFMarkerReader
from llama_index.readers.nougat_ocr import PDFNougatOCR

In [4]:
load_dotenv()
gemini_token = os.getenv("GOOGLE_API_KEY")
llm = GoogleGenAI(model="gemini-2.0-flash", token=gemini_token, max_retries=3,request_timeout=60.0  )
embed_model = GoogleGenAIEmbedding(model="gemini-embedding-exp-03-07", token=gemini_token, max_retries=3, request_timeout=60.0)

reader = WikipediaReader()
documents = reader.load_data(pages=["Bogotazo"])

2025-04-08 11:07:08,929 INFO HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash "HTTP/1.1 200 OK"
HTTP Request: GET https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash "HTTP/1.1 200 OK"


In [5]:
Settings.chunk_size = 300  
Settings.chunk_overlap = 30
Settings.llm = llm 
Settings.embed_model = embed_model

graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

index = KnowledgeGraphIndex.from_documents(documents=documents,
                                           max_triplets_per_chunk=10, 
                                           storage_context=storage_context, 
                                           include_embeddings=True)

2025-04-08 11:07:18,992 INFO AFC is enabled with max remote calls: 10.
AFC is enabled with max remote calls: 10.
2025-04-08 11:07:20,582 INFO HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-04-08 11:07:20,585 INFO AFC remote call 1 is done.
AFC remote call 1 is done.
2025-04-08 11:07:21,337 INFO HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-04-08 11:07:21,414 INFO AFC is enabled with max remote calls: 10.
AFC is enabled with max remote calls: 10.
2025-04-08 11:07:22,526 INFO HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flas

In [6]:
graph = index.get_networkx_graph()
net = Network(notebook=True, cdn_resources='remote', directed=True)
net.from_nx(graph)
net.show("graph.html")
import webbrowser
webbrowser.open("graph.html")

graph.html


True

In [7]:
query_engine = index.as_query_engine(include_text=True, response_mode="tree_summarize", embedding_mode="hybrid", similarity_top_k=5)

try:
	response = query_engine.query("who were the accomplices who help the murder of the person that die in bogotazo?")
	#print(response)
except Exception as e:
	print(f"An error occurred: {e}")

2025-04-08 11:08:16,658 INFO AFC is enabled with max remote calls: 10.
AFC is enabled with max remote calls: 10.
2025-04-08 11:08:17,554 INFO HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-04-08 11:08:17,557 INFO AFC remote call 1 is done.
AFC remote call 1 is done.
2025-04-08 11:08:18,223 INFO HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:batchEmbedContents "HTTP/1.1 200 OK"
2025-04-08 11:08:18,235 INFO > Querying with idx: b869d688-f976-4460-9b23-e91faeef426f: El Bogotazo (from "Bogotá" and the -azo suffix of violent augmentation) was a...
> Querying with idx: b869d688-f976-4460-9b23-e91faeef426f: El B

In [8]:
print(response)

There are theories that the assassination was planned and undertaken by other persons in addition to Juan Roa Sierra, but this has never been corroborated. Some publications have mentioned the government of Mariano Ospina Pérez and sectors of the Liberal party as being involved in the murder.

